In [1]:
# Importing the required libraries
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
# Creating a lemmatizer and reading the intents.json file
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('Intents.json').read())

In [3]:
# Creating empty lists

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

In [4]:
# preprocessing the json data
# tokenization

for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        # tokenize each word
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        
        # add documents in the corpus
        documents.append((word_list, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
print(documents)

[(['google'], 'google'), (['search'], 'google'), (['internet'], 'google'), (['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Namaste'], 'greeting'), (['yo'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Get', 'lost'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['bbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['Tell', 'me', 'a', 'joke'], 'j

In [6]:
# Lemmatize, lower case each word and remove duplicates

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
print(words)

["'m", "'s", '10', '19', 'a', 'age', 'am', 'anyone', 'are', 'ask', 'awesome', 'bad', 'bbye', 'be', 'best', 'bye', 'can', 'contact', 'could', 'covid', 'creator', 'cricket', 'current', 'date', 'day', 'designed', 'developer', 'do', 'doing', 'dumb', 'fine', 'for', 'funny', 'get', 'good', 'goodbye', 'google', 'great', 'haha', 'he', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'hot', 'how', 'i', 'idiot', 'india', 'inspiration', 'inspires', 'internet', 'is', 'it', 'joke', 'karan', 'know', 'later', 'latest', 'laugh', 'lmao', 'lol', 'lost', 'made', 'make', 'malik', 'match', 'me', 'motivates', 'namaste', 'news', 'next', 'nice', 'no', 'nope', 'offered', 'ok', 'old', 'programmed', 'programmer', 'provide', 'question', 'riddle', 'rofl', 'score', 'search', 'see', 'set', 'shut', 'song', 'suggest', 'suggestion', 'sup', 'support', 'talk', 'talking', 'tell', 'temperature', 'ten', 'thank', 'thanks', 'that', 'thats', 'the', 'there', 'till', 'time', 'timer', 'to', 'today', 'top', 'up', 'upto'

In [7]:
# sort classes
classes = sorted(list(set(classes)))
print(classes)

['Identity', 'activity', 'age', 'appreciate', 'contact', 'covid19', 'cricket', 'datetime', 'exclaim', 'goodbye', 'google', 'greeting', 'greetreply', 'haha', 'inspire', 'insult', 'jokes', 'karan', 'news', 'nicetty', 'no', 'options', 'programmer', 'riddle', 'song', 'suggest', 'thanks', 'timer', 'weather', 'whatsup']


In [8]:
# creating a pickle file to store the python objects which we will use while predicting

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [9]:
# Now the Machine learning part....

# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # Lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data created")

Training data created


C:\Users\aryan\AppData\Local\Temp\ipykernel_7312\578497123.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [11]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)
model.save('chatbot.h1', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully")

Epoch 1/200
23/23 [==============================] - 0s 2ms/step - loss: 3.4093 - accuracy: 0.0088
Epoch 2/200
23/23 [==============================] - 0s 2ms/step - loss: 3.3626 - accuracy: 0.0796
Epoch 3/200
23/23 [==============================] - 0s 2ms/step - loss: 3.2794 - accuracy: 0.0708
Epoch 4/200
23/23 [==============================] - 0s 2ms/step - loss: 3.2826 - accuracy: 0.0619
Epoch 5/200
23/23 [==============================] - 0s 2ms/step - loss: 3.1809 - accuracy: 0.1770
Epoch 6/200
23/23 [==============================] - 0s 2ms/step - loss: 3.1105 - accuracy: 0.1239
Epoch 7/200
23/23 [==============================] - 0s 2ms/step - loss: 2.9929 - accuracy: 0.2124
Epoch 8/200
23/23 [==============================] - 0s 2ms/step - loss: 2.9057 - accuracy: 0.2389
Epoch 9/200
23/23 [==============================] - 0s 2ms/step - loss: 2.8448 - accuracy: 0.2035
Epoch 10/200
23/23 [==============================] - 0s 2ms/step - loss: 2.7230 - accuracy: 0.3097
Epoch 11/

23/23 [==============================] - 0s 2ms/step - loss: 0.3667 - accuracy: 0.8850
Epoch 84/200
23/23 [==============================] - 0s 2ms/step - loss: 0.3183 - accuracy: 0.9292
Epoch 85/200
23/23 [==============================] - 0s 2ms/step - loss: 0.2324 - accuracy: 0.9204
Epoch 86/200
23/23 [==============================] - 0s 2ms/step - loss: 0.3511 - accuracy: 0.8938
Epoch 87/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1649 - accuracy: 0.9735
Epoch 88/200
23/23 [==============================] - 0s 2ms/step - loss: 0.2201 - accuracy: 0.9381
Epoch 89/200
23/23 [==============================] - 0s 2ms/step - loss: 0.2697 - accuracy: 0.9115
Epoch 90/200
23/23 [==============================] - 0s 2ms/step - loss: 0.2482 - accuracy: 0.9381
Epoch 91/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1697 - accuracy: 0.9558
Epoch 92/200
23/23 [==============================] - 0s 2ms/step - loss: 0.3346 - accuracy: 0.8938
Epoch 93/200


23/23 [==============================] - 0s 2ms/step - loss: 0.2124 - accuracy: 0.9381
Epoch 165/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1266 - accuracy: 0.9558
Epoch 166/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1469 - accuracy: 0.9381
Epoch 167/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1436 - accuracy: 0.9469
Epoch 168/200
23/23 [==============================] - 0s 2ms/step - loss: 0.0787 - accuracy: 0.9823
Epoch 169/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1066 - accuracy: 0.9735
Epoch 170/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1756 - accuracy: 0.9292
Epoch 171/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1874 - accuracy: 0.9292
Epoch 172/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1449 - accuracy: 0.9469
Epoch 173/200
23/23 [==============================] - 0s 2ms/step - loss: 0.1880 - accuracy: 0.9292
Epoc